### DATASET:
```
./kaggle/input/handwriting-model/
├── x_test_224x224x3.npy(SET OF MATRIX OF IMAGES FOR TESTING PROCESS)
│
├── x_train_224x224x3.npy(SET OF MATRIX OF IMAGES FOR TRAINING PROCESS)
│
├── x_train_label.npy(LABEL TRAINING IMAGES)
│
└── test_id.csv (MORE INFORMATION OF TESTSET_IMAGE)
 
```
Link dataset: https://www.kaggle.com/datasets/phmnhnguyt/handwriting-model/

In [1]:
!rm -r /kaggle/working/*

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import glob
from PIL import Image

## DATA LOADER 


#### Train Images

#### x_train

In [3]:
normalize_x = 224
normalize_y = 224

In [4]:
import numpy as np

# Load the data from the .npy file
x_train = np.load('/kaggle/input/handwriting-model/x_train_224x224x3.npy')

# Now, you can use the 'loaded_data' variable to access the data from the file.


In [5]:
x_train.shape

(5280, 224, 224, 3)

In [6]:
train_label= np.load('/kaggle/input/handwriting-model/x_train_label.npy')
# train_label

In [7]:
y = np.array(train_label)

y = y.reshape(-1, 1)
y.shape

(5280, 1)

In [8]:
x_test = np.load('/kaggle/input/handwriting-model/x_test_224x224x3.npy')

# Now, you can use the 'loaded_data' variable to access the data from the file.


In [9]:
x_test.shape

(1320, 224, 224, 3)

## MODEL BUILDING

In [10]:
# import tensorflow as tf
# import tensorflow.keras.applications.mobilenet_v2 as mobilenet_v2
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
# import tensorflow.keras.optimizers as optimizers

# input_shape = (normalize_x, normalize_y, 3)

# base_model = mobilenet_v2.MobileNetV2(weights='imagenet', include_top=False, input_shape=(normalize_x, normalize_y, 3))

# #resnet.layers[0].trainable = False

# model = Sequential()

# model.add(base_model)
# model.add(GlobalAveragePooling2D())
# model.add(Dense(256, activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(2, activation='softmax'))

# sgd = optimizers.legacy.SGD(learning_rate = 0.001, decay = 3e-6, momentum = 0.85, nesterov = True)
# model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [12]:
import tensorflow as tf
from tensorflow.keras.applications import ConvNeXtXLarge  # You can choose a different variant if needed
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.efficientnet import preprocess_input

# Define input shape
input_shape = (normalize_x, normalize_y, 3)

# Load EfficientNetB0 model with pre-trained weights, excluding the top (classification) layer
base_model = ConvNeXtXLarge(weights='imagenet', include_top=False, input_shape=input_shape, include_preprocessing=True)
for layer in base_model.layers:
    layer.trainable = False
    
# Create your model
model = Sequential()

# Add the EfficientNet base model
model.add(base_model)

# Add GlobalAveragePooling2D layer to reduce spatial dimensions
model.add(GlobalAveragePooling2D())

# Add some fully connected layers for classification
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())

# Output layer with softmax activation for classification (assuming 2 classes)
model.add(Dense(2, activation='sigmoid'))

# Define optimizer (you can adjust the learning rate and other parameters)
# sgd = SGD(learning_rate=0.001, decay=3e-6, momentum=0.85, nesterov=True)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [13]:
# from tensorflow.keras.applications import ResNet50
# from tensorflow.keras.layers import Flatten, Dense, Input
# from tensorflow.keras.models import Model, Sequential
# import tensorflow.keras.optimizers as optimizers

# input_shape = (normalize_x, normalize_y, 3)

# resnet = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape,\
#                  pooling = 'avg')

# #resnet.layers[0].trainable = False
# model = Sequential()

# model.add(resnet)
# model.add(Flatten())
# model.add(Dense(512, activation='relu'))
# model.add(Dense(256, activation='relu'))
# model.add(Dense(2, activation='softmax'))

# sgd = optimizers.legacy.SGD(learning_rate = 0.001, decay = 3e-6, momentum = 0.85, nesterov = True)
# model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

Test information of dataset

In [18]:
print(x_train.shape)
# print(x_test_remainder.shape)

(5280, 224, 224, 3)


In [19]:
y = np.array(train_label)

y = y.reshape(-1, 1)
y.shape

(5280, 1)

In [20]:
import tensorflow as tf
from keras.utils import to_categorical
y = tf.keras.utils.to_categorical(y)
print(y.shape)

(5280, 2)


In [21]:
from sklearn.model_selection import train_test_split

X_train_img, X_dev_img, y_train, y_dev = train_test_split(x_train, y, test_size=0.15)

In [22]:
print(X_train_img.shape)
print(y_train.shape)
print(X_dev_img.shape)
print(y_dev.shape)

(4488, 224, 224, 3)
(4488, 2)
(792, 224, 224, 3)
(792, 2)


## MODEL TRAINING

In [25]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
import os 
# Define an ImageDataGenerator with augmentation settings
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest'
)

# Load and preprocess your original training and validation data (X_train, y_train, X_dev, y_dev)
# Make sure to load and preprocess your data before using them here.

# Create a directory to store augmented images (optional)
if not os.path.exists('augmented_data'):
    os.makedirs('augmented_data')

# Use the data generator to perform augmentation
batch_size = 32
augmented_data_generator = datagen.flow(X_train_img, y_train, batch_size=batch_size)

# Define your model (assuming it has been defined and compiled earlier)
# model = ...

# Define a ModelCheckpoint callback
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

# Define a ModelCheckpoint callback
checkpoint = ModelCheckpoint(
    'best_model_checkpoint.h5',  # Path to save the checkpoint file
    monitor='val_accuracy',  # Metric to monitor (e.g., validation accuracy)
    save_best_only=True,  # Save only the best model
    mode='max',  # Mode can be 'max' for accuracy or 'min' for loss
    verbose=1  # Display messages about the checkpoint
)

# Define a ReduceLROnPlateau callback
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  # Metric to monitor (e.g., validation loss)
    factor=0.5,  # Reduce the learning rate by half when the metric plateaus
    patience=3,  # Number of epochs with no improvement after which learning rate will be reduced
    verbose=1,  # Display messages about learning rate reductions
    min_lr=1e-6  # Minimum learning rate (optional)
)

# Train the model with both callbacks
history = model.fit(
    X_train_img, 
    y_train, 
    validation_data=(X_dev_img, y_dev), 
    epochs=10, 
    batch_size=32,
    callbacks=[checkpoint, reduce_lr]  # Include both callbacks
)

Epoch 1/10
141/141 [==============================] - ETA: 0s - loss: 0.2351 - accuracy: 0.9129
Epoch 1: val_accuracy improved from -inf to 0.94571, saving model to best_model_checkpoint.h5
141/141 [==============================] - 180s 1s/step - loss: 0.2351 - accuracy: 0.9129 - val_loss: 0.1665 - val_accuracy: 0.9457 - lr: 0.0010
Epoch 2/10
141/141 [==============================] - ETA: 0s - loss: 0.1089 - accuracy: 0.9601
Epoch 2: val_accuracy did not improve from 0.94571
141/141 [==============================] - 140s 991ms/step - loss: 0.1089 - accuracy: 0.9601 - val_loss: 0.1420 - val_accuracy: 0.9343 - lr: 0.0010
Epoch 3/10
141/141 [==============================] - ETA: 0s - loss: 0.0680 - accuracy: 0.9759
Epoch 3: val_accuracy did not improve from 0.94571
141/141 [==============================] - 140s 992ms/step - loss: 0.0680 - accuracy: 0.9759 - val_loss: 0.1571 - val_accuracy: 0.9331 - lr: 0.0010
Epoch 4/10
141/141 [==============================] - ETA: 0s - loss: 0.063

In [27]:
# Assuming you have a trained model named 'model'
model.save('model.h5')

## PREDICTING PROCESS

In [28]:
model_path = '/kaggle/working/model.h5'
model.load_weights(model_path)
result = model.predict(x_test)

42/42 [==============================] - 41s 889ms/step


In [29]:
model.evaluate(X_dev_img, y_dev)

25/25 [==============================] - 21s 833ms/step - loss: 0.1916 - accuracy: 0.9356


[0.1916174441576004, 0.935606062412262]

In [30]:
result 

array([[7.7001104e-08, 9.9999988e-01],
       [9.9964476e-01, 1.4372892e-04],
       [9.7863001e-01, 1.8016847e-02],
       ...,
       [9.9992394e-01, 1.5766897e-04],
       [7.6262884e-05, 9.9991751e-01],
       [9.9986780e-01, 3.4663564e-05]], dtype=float32)

In [31]:
result[result >= 0.5] = 1
result[result < 0.5] = 0

In [32]:
result.shape

(1320, 2)

In [33]:
result

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [34]:
# prediction = result.tolist()
# prediction = [int(num[0]) for num in prediction]

In [35]:
# x_test

In [36]:
res = np.array(result)

# Use argmax to get the index of the maximum value in each one-hot encoded row
decoded_labels = np.argmax(res, axis=1)
decoded_labels.shape

(1320,)

In [37]:
(decoded_labels)

array([1, 0, 0, ..., 0, 1, 0])

In [38]:
# id = test_id_1 + test_id_unknown
# len(id)

In [39]:
# label_1 = [1] * len(test_id_1)
# label = label_1 + list(decoded_labels)
# # label
test_id = pd.read_csv('/kaggle/input/handwriting-model/test_id.csv').values.reshape(1,-1)[0]
test_id

array([4945787026,  949397428, 4927867913, ..., 2910531219, 5126069048,
       5274253035])

In [40]:

result_df = pd.DataFrame({'id':test_id,'label':decoded_labels})
result_df

,id,label
0,4945787026,1
1,949397428,0
2,4927867913,0
3,6933529726,0
4,5285787627,1
...,...,...
1315,1801276133,0
1316,624050167,1
1317,2910531219,0
1318,5126069048,1


In [41]:
result_df.to_csv('submit.csv', index=False)

File "submit.csv" is the result of prediction of model, for testing image.

In [42]:
(result_df['label'] == 1).sum()/result_df.shape[0]

0.3643939393939394